In [52]:
from telethon.sync import TelegramClient
import pandas as pd
from hazm import Normalizer, WordTokenizer, stopwords_list, Lemmatizer
import re
import emoji
from gensim.models.phrases import Phrases, Phraser
from tqdm.auto import tqdm
from sentence_transformers import SentenceTransformer
from bertopic import BERTopic
from sklearn.feature_extraction.text import CountVectorizer
from umap import UMAP
from hdbscan import HDBSCAN
from os import getenv
from dotenv import load_dotenv


load_dotenv()

api_id = int(getenv("TG_API_ID"))          
api_hash = getenv("TG_API_HASH")

client = TelegramClient("session_persian", api_id, api_hash)


In [53]:
await client.start()

In [54]:
# Scraping for Data from a Telgram channel
channel_username = 'IranintlTV'   
limit = 10000                     
keywords = ['جنگ']  

all_messages = []

async with client:
    async for message in client.iter_messages(channel_username, limit=limit):
        if message.message:
            text = message.message
            if any(keyword in text for keyword in keywords):
                all_messages.append({
                    "date": message.date,
                    "text": text
                })

df = pd.DataFrame(all_messages)
df.to_csv("../data/data.csv", index=False, encoding="utf-8-sig")
df.head()

,date,text
0,2025-07-23 01:37:17+00:00,حسن خمینی، نوه رهبر پیشین جمهوری اسلامی گفت: ا...
1,2025-07-22 23:51:14+00:00,حسین دارابی کارگردان فیلم خدای جنگ در یک مصاحب...
2,2025-07-22 23:17:31+00:00,کریس رایت، وزیر انرژی آمریکا، روز سه‌شنبه در گ...
3,2025-07-22 23:06:47+00:00,مراد ویسی، تحلیل‌گر ارشد ایران‌اینترنشنال، می‌...
4,2025-07-22 22:19:36+00:00,🎧نسخه صوتی سیاست با مراد ویسی: جنگ سوم جمهوری...


In [55]:
# Cleaning the text for the embedder and the tokenizer
def clean_text(text):
    if not isinstance(text, str):
        return ""
    text = emoji.replace_emoji(text, replace='')         
    text = re.sub(r"@\w+", "", text)                     
    text = re.sub(r"http\S+|www\S+", "", text)           
    text = re.sub(r"\s+", " ", text).strip()             
    return text

In [56]:
df = pd.read_csv("../data/data.csv")
df["clean_text"] = df["text"].map(clean_text)
df[['text', 'clean_text']].head()

,text,clean_text
0,حسن خمینی، نوه رهبر پیشین جمهوری اسلامی گفت: ا...,حسن خمینی، نوه رهبر پیشین جمهوری اسلامی گفت: ا...
1,حسین دارابی کارگردان فیلم خدای جنگ در یک مصاحب...,حسین دارابی کارگردان فیلم خدای جنگ در یک مصاحب...
2,کریس رایت، وزیر انرژی آمریکا، روز سه‌شنبه در گ...,کریس رایت، وزیر انرژی آمریکا، روز سه‌شنبه در گ...
3,مراد ویسی، تحلیل‌گر ارشد ایران‌اینترنشنال، می‌...,مراد ویسی، تحلیل‌گر ارشد ایران‌اینترنشنال، می‌...
4,🎧نسخه صوتی سیاست با مراد ویسی: جنگ سوم جمهوری...,نسخه صوتی سیاست با مراد ویسی: جنگ سوم جمهوری‌ا...


In [57]:
# A light tokenizer for generating bigrams 
light_stops = {"است", "می", "شود"}
normalizer = Normalizer(
    persian_numbers=True,
    remove_diacritics=True,
)
tokenizer = WordTokenizer()
lemmatizer  = Lemmatizer()

def persian_tokenizer_light(text: str) -> list[str]:
    if not isinstance(text, str):
        return []
    text = normalizer.normalize(text)
    text = re.sub(r"[^\u0600-\u06FF0-9\s]", " ", text)

    tokens = [
        lemmatizer.lemmatize(tok)
        for tok in tokenizer.tokenize(text)
        if tok not in light_stops and len(tok) > 1
    ]
    cleaned = []
    for t in tokens:
        if "#" in t:           
            continue
        if t.isdigit():        
            continue
        if len(t) < 3:         
            continue
        cleaned.append(t)
    return cleaned

In [58]:
class SentenceIterator:
    def __init__(self, series):
        self.series = series  
    def __iter__(self):
        for txt in self.series:
            yield persian_tokenizer_light(txt)

sentences = SentenceIterator(df['clean_text'])

In [59]:
# Generating bigrams 
cached = list(tqdm(sentences, desc="Tokenising"))

bigram_model = Phrases(
    cached,
    min_count=10,
    threshold=5,
    #scoring='npmi',
    delimiter='_',      
)
bigram_phraser = Phraser(bigram_model)
bigram_phraser.save("persian_bigram_phraser.gensim")


sentences_bigrams = (bigram_phraser[sent] for sent in cached)   

Tokenising: 1525it [00:01, 1246.20it/s]


In [60]:
# Final Tokenization
base_stops = set(stopwords_list())
extra_stops = extra_stops = {
    "است","می","شود","iranintltv","http","https",
    "jpg","png","video","photo","کانال","تلگرام","ای","های","ها",
    "ایراناینترنشنال", "اینترنشنال", "ویدیو", "کانال"
}

stops = base_stops | extra_stops

bigram_phraser  = Phraser.load("persian_bigram_phraser.gensim")

def persian_tokenizer(text: str) -> list[str]:
    if not isinstance(text, str):
        return []
    
    tokens = persian_tokenizer_light(text)          
    tokens = bigram_phraser[tokens]                 

    tokens = [
        t for t in tokens
        if t not in stops       
        and len(t) >= 3
        and not t.isdigit()
    ]

    return tokens

df['tokens'] = df['clean_text'].apply(persian_tokenizer)
df[['clean_text', 'tokens']].head()

,clean_text,tokens
0,حسن خمینی، نوه رهبر پیشین جمهوری اسلامی گفت: ا...,"[حسن, خمینی, نوه, رهبر, پیشین, جمهوری_اسلامی, ..."
1,حسین دارابی کارگردان فیلم خدای جنگ در یک مصاحب...,"[حسین, دارابی, کارگردان, فیلم, خدای, جنگ, مصاح..."
2,کریس رایت، وزیر انرژی آمریکا، روز سه‌شنبه در گ...,"[کریس, رایت, وزیر, انرژی, آمریکا, روز_شنبه, فا..."
3,مراد ویسی، تحلیل‌گر ارشد ایران‌اینترنشنال، می‌...,"[مراد_ویسی, تحلیل_ارشد, ایران_اینترنشنال, اسرا..."
4,نسخه صوتی سیاست با مراد ویسی: جنگ سوم جمهوری‌ا...,"[نسخه_صوتی, سیاست, مراد_ویسی, جنگ, جمهوری_اسلا..."


In [61]:
sample_doc = df['clean_text'].iloc[0]
print("Original:", sample_doc[:120], "...")
print("Tokens:", persian_tokenizer_light(sample_doc)[:15])
print("With bigrams:", bigram_phraser[persian_tokenizer_light(sample_doc)][:15])

Original: حسن خمینی، نوه رهبر پیشین جمهوری اسلامی گفت: امروز که به دوگانه «وطن‌دوست و وطن‌فروش» رسیده‌ایم، همه خودی‌ها کسانی‌اند ک ...
Tokens: ['حسن', 'خمینی', 'نوه', 'رهبر', 'پیشین', 'جمهوری', 'اسلامی', 'امروز', 'دوگانه', 'وطن', 'دوست', 'وطن', 'فروش', 'همه', 'خودی']
With bigrams: ['حسن', 'خمینی', 'نوه', 'رهبر', 'پیشین', 'جمهوری_اسلامی', 'امروز', 'دوگانه', 'وطن', 'دوست', 'وطن', 'فروش', 'همه', 'خودی', 'کسانی']


In [62]:
df.to_csv("../preprocess/predata.csv", index=False, encoding="utf-8-sig")

In [63]:
df = pd.read_csv("../preprocess/predata.csv")

df=df[df['clean_text'].str.len() > 20]
df = df.drop_duplicates("clean_text") 
docs = df['clean_text'].dropna().astype(str).tolist()

In [64]:
# Sentence embeddings 
embedder = SentenceTransformer(
    "xmanii/maux-gte-persian",
    trust_remote_code=True
)

# Vectorizing 
vectorizer = CountVectorizer(
    tokenizer=persian_tokenizer,   
    token_pattern=None,            
    lowercase=False       
)

# Clustering knobs
umap_model = UMAP(n_neighbors=15, min_dist=0.1, metric="cosine")
hdb_model  = HDBSCAN(min_cluster_size=10, min_samples=5, prediction_data=True)

# Fit
topic_model = BERTopic(
    embedding_model=embedder,
    vectorizer_model=vectorizer,
    umap_model=umap_model,
    hdbscan_model=hdb_model,
    min_topic_size=10,
    calculate_probabilities=True,
)

topics, probs = topic_model.fit_transform(docs)
topic_model.get_topic_info().head(10)

,Topic,Count,Name,Representation,Representative_Docs
0,-1,301,-1_اسرائیل_جنگ_جمهوری_اسلامی_ایران,"[اسرائیل, جنگ, جمهوری_اسلامی, ایران, حمله, آمر...",[ساکنان شهر عمدتا دروزی‌نشین سویدا در جنوب سور...
1,0,212,0_جمهوری_اسلامی_اسرائیل_جنگ_ایران_اینترنشنال,"[جمهوری_اسلامی, اسرائیل, جنگ, ایران_اینترنشنال...",[مراد ویسی، تحلیل‌گر ایران‌اینترنشنال گفت: «با...
2,1,142,1_ترامپ_آمریکا_دونالد_ترامپ_رییس_جمهوری,"[ترامپ, آمریکا, دونالد_ترامپ, رییس_جمهوری, ایر...",[دونالد ترامپ، رییس‌جمهوری آمریکا، اعلام کرد ک...
3,2,55,2_دشمن_جنگ_روزه_اشاره_جنگ_محمدباقر_قالیباف,"[دشمن, جنگ_روزه, اشاره_جنگ, محمدباقر_قالیباف, ...",[مسعود پزشکیان با انتشار پیامی خطاب به مردم گف...
4,3,50,3_ارتش_اسرائیل_جنگنده_هدف_قرار_نیروی_هوایی,"[ارتش_اسرائیل, جنگنده, هدف_قرار, نیروی_هوایی, ...",[به‌گفته ارتش اسرائیل، بامداد یکشنبه و با هدای...
5,4,50,4_خامنه_علی_خامنه_پیام_رهبر_جمهوری,"[خامنه, علی_خامنه, پیام, رهبر_جمهوری, یاد, اسل...",[خامنه‌ای از مخفیگاهش به مردم پیام داد که «جنگ...
6,5,46,5_قطع_برق_ارسال_ویدیو_اینترنت,"[قطع, برق, ارسال_ویدیو, اینترنت, ایران_اینترنش...",[یکی از شهروندان با ارسال ویدیویی به ایران‌این...
7,6,39,6_وزیر_خارجه_عباس_عراقچی_جمهوری_اسلامی_ادامه,"[وزیر_خارجه, عباس_عراقچی, جمهوری_اسلامی, ادامه...",[عباس عراقچی، وزیر خارجه جمهوری اسلامی، در مصا...
8,7,38,7_اقتصادی_بورس_اقتصاد_افزایش,"[اقتصادی, بورس, اقتصاد, افزایش, قیمت, کاهش, با...",[کارشناسان از پیامدهای بحران اقتصادی پس از جنگ...
9,8,37,8_فرمانده_سپاه_دشمن_نیرو_مسلح_مشاور,"[فرمانده_سپاه, دشمن, نیرو_مسلح, مشاور, فرمانده...",[ابراهیم جباری، مشاور فرمانده کل سپاه پاسداران...


In [ ]:
topic_model.get_topic(0)

[('جمهوری_اسلامی', 0.03078131583108451),
 ('اسرائیل', 0.02831422829534644),
 ('جنگ', 0.028142982331857528),
 ('ایران_اینترنشنال', 0.02150137329504017),
 ('ایران', 0.018986527099341635),
 ('مردم', 0.016611496780444008),
 ('حکومت', 0.0164050533350266),
 ('مراد_ویسی', 0.015316669409790231),
 ('تحلیل_ارشد', 0.01426373549939889),
 ('نظامی', 0.013248543786430101)]

In [66]:
topic_model.visualize_topics()
